# データ整形

In [ ]:
import pandas as pd
from glob import glob

In [ ]:
# フォルダ内のファイル達を取得

filepaths_order = glob('sources/order_new/order*.xlsx')
filepaths_order

['sources/order_new/order_C_20210203.xlsx',
 'sources/order_new/order_D_20210203.xlsx',
 'sources/order_new/order_A_20210203.xlsx',
 'sources/order_new/order_B_20210203.xlsx']

In [ ]:
filepath = filepaths_order[0]
filepath

In [ ]:
# excelファイルをDF化

df_order = pd.read_excel(filepath)
df_order

,焼酎,白ワイン,ブランデー
0,8,18,2


In [ ]:
# DFから必要部分を抽出して、Dictとして情報をまとめたい

df_order.to_dict().items()

dict_items([('焼酎', {0: 8}), ('白ワイン', {0: 18}), ('ブランデー', {0: 2})])

In [ ]:
from collections import defaultdict

In [ ]:
# 情報まとめる用のDict用意。acc

order = defaultdict(int)

In [ ]:
# valueがdictになっていて、さらにそのvalue部分の値を取得するには？

for key, value in df_order.to_dict().items():
    order[key] += value[0]
    print(value[0])

8
18
2


In [ ]:
# acc用dictに情報集まる

order

defaultdict(int, {'焼酎': 8, '白ワイン': 18, 'ブランデー': 2})

In [ ]:
filepath = filepaths_order[1]
df_order = pd.read_excel(filepath)
df_order

,焼酎,赤ワイン,白ワイン
0,3,4,6


In [ ]:
for key, value in df_order.to_dict().items():
    order[key] += value[0]

3
4
6


In [ ]:
order

defaultdict(int, {'焼酎': 11, '白ワイン': 24, 'ブランデー': 2, '赤ワイン': 4})

# 💡 集計済みファイルを別フォルダに移動

In [ ]:
import os
import shutil

In [ ]:
os.listdir('sources/order_new/')

['order_C_20210203.xlsx',
 'order_D_20210203.xlsx',
 'order_A_20210203.xlsx',
 'order_B_20210203.xlsx']

In [ ]:
os.listdir('sources/order_old/')

['.DS_Store']

In [ ]:
filepath = filepaths_order[0]
shutil.move(filepath, 'sources/order_old/')

'sources/order_old/order_C_20210203.xlsx'

## 今までの流れを関数化

In [ ]:
def update_order(order, filepath):
    # 注文表読み込み
    df_order = pd.read_excel(filepath)
    for key, value in df_order.to_dict().items():
        order[key] += value[0]
    # ファイルの移動
    shutil.move(filepath, 'sources/order_old/')
    return order

In [ ]:
# フォルダ内のすべてのファイルを集計する。

order = defaultdict(int)

for filepath in filepaths_order:
    order = update_order(order, filepath)

In [ ]:
order

defaultdict(int,
            {'焼酎': 16.0,
             '白ワイン': 54.0,
             'ブランデー': 2,
             '赤ワイン': 4,
             'ウィスキー': 7.0,
             '缶チューハイ': 23,
             'Unnamed: 3': nan,
             'Unnamed: 4': nan,
             'Unnamed: 5': nan,
             'Unnamed: 6': nan,
             'Unnamed: 7': nan,
             'Unnamed: 8': nan})

In [ ]:
# 在庫表をDF化

filepath_stock = 'sources/stock.xlsx'
df_stock = pd.read_excel(filepath_stock)

df_stock.tail(3)

,日付,曜日,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
10,2021-02-04,Thu,99,66,77,107,31,37,149
11,2021-02-05,Fri,78,62,55,92,41,30,121
12,2021-02-06,Sat,66,48,38,70,31,18,98


#### Tips: DF同士の計算

In [ ]:
pd.DataFrame([[10, 4, 6]])

,0,1,2
0,10,4,6


In [ ]:
pd.DataFrame([[5, 1, 10]])

,0,1,2
0,5,1,10


In [ ]:
pd.DataFrame([[10, 4, 6]]) - pd.DataFrame([[5, 1, 10]])

,0,1,2
0,5,3,-4


In [ ]:
# DFの必要な行・列のみ抽出

stock = df_stock.iloc[-1:, 2:]
stock

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
12,66,48,38,70,31,18,98


In [ ]:
# 元のorderはdefaultdict型なので、DF変換
# ただしこのままだと列の順番バラバラ・いらない列入り込んでいる。。。

order = pd.DataFrame(order, index=[0])
order

,焼酎,白ワイン,ブランデー,赤ワイン,ウィスキー,缶チューハイ,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,16.0,54.0,2,4,7.0,23,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 計算相手のカラム構造似合わせる

order = pd.DataFrame(order, index=[0], columns=stock.columns)

In [ ]:
order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,NaN,16.0,4,54.0,7.0,2,23


In [ ]:
# NaNを0で補間する

order = order.fillna(0)
order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,0.0,16.0,4,54.0,7.0,2,23


In [ ]:
# 計算してみると、変な結果になる。。。
# 行番号がおかしいから

stock - order

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 行番号リセット

stock = stock.reset_index(drop=True)

In [ ]:
# ようやく計算できる
# updated :
#   <DF>
#   各店舗に発送したあとの在庫数

updated = stock - order
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,66.0,32.0,34,16.0,24.0,16,75


# 注文が必要な商品を洗い出し

In [ ]:
# masterに、閾値・追加量情報が記載してある
# 読み込んでDF

filepath_master = 'sources/master.xlsx'
df_master = pd.read_excel(filepath_master)
df_master

,Unnamed: 0,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,閾値,30,20,20,20,30,10,40
1,追加量,100,80,100,100,80,80,120


In [ ]:
threshold = df_master.iloc[:1 , 1:]
threshold

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,30,20,20,20,30,10,40


In [ ]:
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,66.0,32.0,34,16.0,24.0,16,75


In [ ]:
# 閾値を下回っている商品を抽出して、
# NaNを削除して
# 商品名だけを取得する

shortage_columns = updated[updated < threshold].dropna(axis=1).columns
shortage_columns

Index(['白ワイン', 'ウィスキー'], dtype='object')

In [ ]:
# masterから不足商品の追加量を取得する

df_shortage = df_master.iloc[1:, 1:][shortage_columns]
df_shortage

,白ワイン,ウィスキー
1,100,80


# メール作成

In [ ]:
order_text = ''

for key, value in df_shortage.to_dict().items():
    order_text += f'{key}を{value[1]}本、'

In [ ]:
order_text

'白ワインを100本、ウィスキーを80本、'

---

下記ページでGmailのセキュリティ周りを変更  
https://myaccount.google.com/u/2/security

```json
{
    "ADDRESS": "メールアドレス",
    "PASSWORD": "セキュリティ画面で発行したパスワード",    
}
```

In [ ]:
import json
with open('sources/secret.json') as f:
    address_password = json.load(f)

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.utils import formatdate

from_addr = address_password['ADDRESS']
password = address_password['PASSWORD']

subject = '発注依頼'
body = f'○○さん、在庫数が足りなくなってしまったため、{order_text}発注してください。'
to_addr = ''

# SMTPサーバに接続
smtpobj = smtplib.SMTP('smtp.gmail.com', 587)
smtpobj.starttls()
smtpobj.login(from_addr, password)

# メッセージ（メール）の作成
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = from_addr
msg['To'] = to_addr
msg['Date'] = formatdate()

# 作成したメールを送信
smtpobj.send_message(msg)
smtpobj.close()

# 最新の在庫情報の反映

In [ ]:
from datetime import date

In [ ]:
today = date.today()

In [ ]:
# 曜日の取得

print(today.strftime('%a'))

Sun


In [ ]:
updated['日付'] = today
updated['曜日'] = today.strftime('%a')

In [ ]:
updated

,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ,日付,曜日
0,66.0,32.0,34,16.0,24.0,16,75,2021-02-07,Sun


In [ ]:
pd.concat([df_stock, updated], sort=False).reset_index(drop=True).to_excel(filepath_stock, index=False)

In [ ]:
pd.read_excel(filepath_stock)

,日付,曜日,日本酒,焼酎,赤ワイン,白ワイン,ウィスキー,ブランデー,缶チューハイ
0,2021-01-25,Mon,100,80,51,63,150,60,95
1,2021-01-26,Tue,88,73,35,38,130,50,82
2,2021-01-27,Wed,73,66,105,15,126,37,75
3,2021-01-28,Thu,50,55,96,123,114,27,43
4,2021-01-29,Fri,39,44,75,96,106,13,19
5,2021-01-30,Sat,25,31,64,88,98,10,117
6,2021-01-31,Sun,125,106,37,77,83,84,97
7,2021-02-01,Mon,120,96,34,50,69,75,67
8,2021-02-02,Tue,112,85,10,35,54,53,49
9,2021-02-03,Wed,103,74,90,19,41,44,12


# 一連の流れをスクリプト化

In [ ]:
import pandas as pd
from glob import glob
from collections import defaultdict

import os
import shutil

import json    
import smtplib
from email.mime.text import MIMEText
from email.utils import formatdate
from datetime import date

# 注文表の集計
def update_order(order, filepath):
    # 注文表読み込み
    df_order = pd.read_excel(filepath)
    for key, value in df_order.to_dict().items():
        order[key] += value[0]
    # ファイルの移動
    shutil.move(filepath, 'sources/order_old/')
    return order

def main():
    print('ーーーーー開始ーーーーー')
    # 注文情報の収集
    print('注文情報の収集')
    order = defaultdict(int)
    filepaths_order = glob('sources/order_new/order*.xlsx')
    for filepath in filepaths_order:
        order = update_order(order, filepath)
    print('Done')

    # 最新の在庫量の確認
    print('最新の在庫量の確認')
    filepath_stock = 'sources/stock.xlsx'
    df_stock = pd.read_excel(filepath_stock)

    stock = df_stock.iloc[-1:, 2:]
    stock = stock.reset_index(drop=True)

    order = pd.DataFrame(order, index=[0], columns=stock.columns)
    order = order.fillna(0)

    updated = stock - order
    print('Done')


    # 追加注文品の算出
    print('追加注文品の算出')
    filepath_master = 'sources/master.xlsx'
    df_master = pd.read_excel(filepath_master)

    threshold = df_master.iloc[:1 , 1:]
    shortage_columns = updated[updated < threshold].dropna(axis=1).columns

    df_shortage = df_master.iloc[1:, 1:][shortage_columns]

    order_text = ''
    for key, value in df_shortage.to_dict().items():
        order_text += f'{key}を{value[1]}本、'
    print('Done')


    # Gmailで注文
    print('Gmailで注文')
    with open('sources/secret.json') as f:
        address_password = json.load(f)

    from_addr = address_password['ADDRESS']
    password = address_password['PASSWORD']

    subject = '発注依頼'
    body = f'○○さん、在庫数が足りなくなってしまったため、{order_text}発注してください。'
    to_addr = ''

    # SMTPサーバに接続
    smtpobj = smtplib.SMTP('smtp.gmail.com', 587)
    smtpobj.starttls()
    smtpobj.login(from_addr, password)

    # メッセージ（メール）の作成
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_addr
    msg['To'] = to_addr
    msg['Date'] = formatdate()

    # 作成したメールを送信
    smtpobj.send_message(msg)
    smtpobj.close()
    print('Done')


    # 在庫表の更新
    print('在庫表の更新')
    today = date.today()
    updated['日付'] = today
    updated['曜日'] = today.strftime('%a')

    pd.concat([df_stock, updated], sort=False).reset_index(drop=True).to_excel(filepath_stock, index=False)
    pd.read_excel(filepath_stock)
    print('ーーーーー終了ーーーーー')

if __name__ == '__main__':
    main()